# 🚕 NYC Taxi Dashboard Métier

Dashboard interactif pour l'analyse des données transformées (tables FINAL créées par dbt).

In [4]:
# Imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import snowflake.connector
from dotenv import load_dotenv
import os

# Configuration
load_dotenv()
px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = "viridis"

In [5]:
# Connexion Snowflake
def get_snowflake_connection():
    return snowflake.connector.connect(
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        warehouse="NYC_TAXI_WH",
        database="NYC_TAXI_DB",
        schema="RAW",  # Les tables dbt sont créées dans RAW par défaut
        role="NYCTRANSFORM"
    )

def query_to_df(query):
    """Exécuter une requête et retourner un DataFrame"""
    conn = get_snowflake_connection()
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
        return pd.DataFrame(data, columns=columns)
    finally:
        conn.close()

print("✅ Fonctions de connexion définies")

✅ Fonctions de connexion définies


In [6]:
# Chargement des données depuis les tables dbt
print("📊 Chargement des données depuis les tables dbt...")

# Données quotidiennes (table dbt)
daily_data = query_to_df("""
    SELECT pickup_date, total_trips, total_revenue, avg_distance, avg_tip_percentage
    FROM daily_summary 
    ORDER BY pickup_date
""")

# Données horaires (table dbt)
hourly_data = query_to_df("""
    SELECT pickup_hour, total_trips, avg_revenue, avg_speed, time_period
    FROM hourly_patterns 
    ORDER BY pickup_hour
""")

# Top zones (table dbt)
zone_data = query_to_df("""
    SELECT pickup_zone, total_trips, total_revenue, avg_distance, popularity_rank
    FROM zone_analysis 
    ORDER BY total_trips DESC 
    LIMIT 20
""")

# Catégories de trajets (table dbt)
category_data = query_to_df("""
    SELECT distance_category, day_type, total_trips, avg_revenue, avg_duration, avg_speed
    FROM trip_categories 
    ORDER BY total_trips DESC
""")

print(f"✅ Données dbt chargées:")
print(f"   - Daily: {len(daily_data):,} jours")
print(f"   - Hourly: {len(hourly_data)} heures")
print(f"   - Zones: {len(zone_data)} zones")
print(f"   - Categories: {len(category_data)} catégories")

📊 Chargement des données depuis les tables dbt...


InterfaceError: 252005: Failed to convert current row, cause: ordinal must be >= 1

## 📈 Dashboard Interactif

In [ ]:
# Widgets de contrôle
date_range = widgets.SelectionRangeSlider(
    options=daily_data['PICKUP_DATE'].dt.strftime('%Y-%m-%d').tolist()[::30],  # Tous les 30 jours
    index=(0, -1),
    description='Période:',
    layout=widgets.Layout(width='500px')
)

metric_selector = widgets.Dropdown(
    options=[('Nombre de trajets', 'TOTAL_TRIPS'), 
             ('Revenus totaux', 'TOTAL_REVENUE'),
             ('Distance moyenne', 'AVG_DISTANCE'),
             ('Pourboire moyen', 'AVG_TIP_PERCENTAGE')],
    value='TOTAL_TRIPS',
    description='Métrique:'
)

zone_count = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    description='Top zones:'
)

display(widgets.HBox([date_range, metric_selector, zone_count]))

In [ ]:
# Fonction de mise à jour des graphiques
def update_dashboard():
    # Filtrer les données selon la période sélectionnée
    start_date, end_date = date_range.value
    filtered_daily = daily_data[
        (daily_data['PICKUP_DATE'] >= start_date) & 
        (daily_data['PICKUP_DATE'] <= end_date)
    ]
    
    # Créer les sous-graphiques
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'Évolution {metric_selector.label}',
            'Patterns Horaires',
            f'Top {zone_count.value} Zones',
            'Catégories de Trajets'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "pie"}]]
    )
    
    # 1. Évolution temporelle
    fig.add_trace(
        go.Scatter(
            x=filtered_daily['PICKUP_DATE'],
            y=filtered_daily[metric_selector.value],
            mode='lines+markers',
            name=metric_selector.label,
            line=dict(color='#1f77b4', width=2)
        ),
        row=1, col=1
    )
    
    # 2. Patterns horaires
    colors = {'Rush Matinal': '#ff7f0e', 'Journée': '#2ca02c', 
              'Rush Soir': '#d62728', 'Soirée': '#9467bd', 'Nuit': '#8c564b'}
    
    fig.add_trace(
        go.Bar(
            x=hourly_data['PICKUP_HOUR'],
            y=hourly_data['TOTAL_TRIPS'],
            name='Trajets/heure',
            marker_color=[colors.get(period, '#1f77b4') for period in hourly_data['TIME_PERIOD']]
        ),
        row=1, col=2
    )
    
    # 3. Top zones
    top_zones = zone_data.head(zone_count.value)
    fig.add_trace(
        go.Bar(
            x=top_zones['TOTAL_TRIPS'],
            y=[f"Zone {zone}" for zone in top_zones['PICKUP_ZONE']],
            orientation='h',
            name='Trajets par zone',
            marker_color='#2ca02c'
        ),
        row=2, col=1
    )
    
    # 4. Catégories (pie chart)
    fig.add_trace(
        go.Pie(
            labels=category_data['DISTANCE_CATEGORY'],
            values=category_data['TOTAL_TRIPS'],
            name="Catégories"
        ),
        row=2, col=2
    )
    
    # Mise en forme
    fig.update_layout(
        height=800,
        title_text="🚕 NYC Taxi Dashboard - Données Transformées (dbt)",
        title_x=0.5,
        showlegend=False
    )
    
    # Axes
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Heure", row=1, col=2)
    fig.update_xaxes(title_text="Nombre de trajets", row=2, col=1)
    
    fig.update_yaxes(title_text=metric_selector.label, row=1, col=1)
    fig.update_yaxes(title_text="Trajets", row=1, col=2)
    fig.update_yaxes(title_text="Zone", row=2, col=1)
    
    return fig

# Affichage initial
dashboard_output = widgets.Output()

def on_change(change):
    with dashboard_output:
        dashboard_output.clear_output(wait=True)
        fig = update_dashboard()
        fig.show()

# Lier les widgets aux mises à jour
date_range.observe(on_change, names='value')
metric_selector.observe(on_change, names='value')
zone_count.observe(on_change, names='value')

# Affichage initial
with dashboard_output:
    fig = update_dashboard()
    fig.show()

display(dashboard_output)

## 📊 KPIs Métier (depuis dbt)

In [ ]:
# Calcul des KPIs depuis les tables dbt
total_trips = daily_data['TOTAL_TRIPS'].sum()
total_revenue = daily_data['TOTAL_REVENUE'].sum()
avg_trip_revenue = total_revenue / total_trips
avg_distance = daily_data['AVG_DISTANCE'].mean()
top_zone = zone_data.iloc[0]['PICKUP_ZONE']
peak_hour = hourly_data.loc[hourly_data['TOTAL_TRIPS'].idxmax(), 'PICKUP_HOUR']

# Affichage des KPIs
kpi_html = f"""
<div style="display: flex; justify-content: space-around; background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <div style="text-align: center;">
        <h3 style="color: #2c3e50; margin: 0;">{total_trips:,.0f}</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Total Trajets (dbt)</p>
    </div>
    <div style="text-align: center;">
        <h3 style="color: #27ae60; margin: 0;">${total_revenue:,.0f}</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Revenus Totaux</p>
    </div>
    <div style="text-align: center;">
        <h3 style="color: #3498db; margin: 0;">${avg_trip_revenue:.2f}</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Revenu Moyen/Trajet</p>
    </div>
    <div style="text-align: center;">
        <h3 style="color: #e74c3c; margin: 0;">{avg_distance:.1f} mi</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Distance Moyenne</p>
    </div>
    <div style="text-align: center;">
        <h3 style="color: #9b59b6; margin: 0;">Zone {top_zone}</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Zone la Plus Active</p>
    </div>
    <div style="text-align: center;">
        <h3 style="color: #f39c12; margin: 0;">{peak_hour}h</h3>
        <p style="color: #7f8c8d; margin: 5px 0;">Heure de Pointe</p>
    </div>
</div>
"""

display(HTML(kpi_html))

## 🎯 Insights Métier (dbt)

In [ ]:
# Analyse des insights depuis les données dbt
insights = []

# Analyse temporelle
busiest_month = daily_data.groupby(daily_data['PICKUP_DATE'].dt.to_period('M'))['TOTAL_TRIPS'].sum().idxmax()
insights.append(f"📅 **Mois le plus actif**: {busiest_month}")

# Analyse des revenus
best_revenue_zone = zone_data.loc[zone_data['TOTAL_REVENUE'].idxmax(), 'PICKUP_ZONE']
insights.append(f"💰 **Zone la plus rentable**: Zone {best_revenue_zone}")

# Analyse des distances (depuis dbt)
if not category_data.empty:
    long_trips_pct = (category_data[category_data['DISTANCE_CATEGORY'].isin(['Longs trajets', 'Très longs trajets'])]['TOTAL_TRIPS'].sum() / 
                      category_data['TOTAL_TRIPS'].sum() * 100)
    insights.append(f"🚗 **Trajets longs (>5mi)**: {long_trips_pct:.1f}% du total")

# Analyse weekend vs semaine (depuis dbt)
weekend_data = category_data[category_data['DAY_TYPE'] == 'Weekend']
weekday_data = category_data[category_data['DAY_TYPE'] == 'Jours de semaine']
if not weekend_data.empty and not weekday_data.empty:
    weekend_avg = weekend_data['AVG_REVENUE'].mean()
    weekday_avg = weekday_data['AVG_REVENUE'].mean()
    diff_pct = ((weekend_avg - weekday_avg) / weekday_avg * 100)
    insights.append(f"📊 **Weekend vs Semaine**: {diff_pct:+.1f}% de revenus en weekend")

# Affichage des insights
insights_html = "<div style='background: #e8f5e8; padding: 15px; border-radius: 8px; margin: 10px 0;'>\n"
insights_html += "<h4 style='color: #2c3e50; margin-top: 0;'>🎯 Insights Métier (Tables dbt)</h4>\n"
for insight in insights:
    insights_html += f"<p style='margin: 8px 0; color: #2c3e50;'>{insight}</p>\n"
insights_html += "</div>"

display(HTML(insights_html))

print("✅ Dashboard métier dbt prêt !")
print("🎮 Utilisez les widgets ci-dessus pour explorer les données transformées")
print("📊 Données sources: Tables dbt (daily_summary, zone_analysis, hourly_patterns, trip_categories)")